<a href="https://colab.research.google.com/github/takuyats/ngboost/blob/master/%E6%96%87%E6%9B%B8%E5%88%86%E9%A1%9E%E3%81%A7%E8%87%AA%E7%84%B6%E8%A8%80%E8%AA%9E%E5%87%A6%E7%90%86%E3%81%AB%E8%A7%A6%E3%82%8C%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 文書分類で自然言語処理に触れる
- 高橋寛治 [@kanji250tr](https://twitter.com/kanji250tr)
- [スライド資料リンク](https://docs.google.com/presentation/d/e/2PACX-1vQHjfOTrH-xu49F6iINswP99AedOSgkYLlc4smMkkCi-GJ1WrA3CKAPr8epiRR8Cy4t021tP_6MWf7f/pub?start=false&loop=false&delayms=3000)

## 概要
本資料は、Pythonを用いて文書分類を行うことで、自然言語処理に触れることを目的としています。

## 環境構築
- 形態素解析器MeCabのインストール
- livedoorニュースコーパスの入手

### 形態素解析器のインストール

In [0]:
!apt-get -y install mecab libmecab-dev mecab-ipadic-utf8 > /dev/null
!pip install mecab-python3 > /dev/null

### livedoorニュースコーパスの取得と整理
wgetコマンドによりダウンロードしたものを展開する。

README.txtを表示する。

In [0]:
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz

--2018-07-31 11:09:58--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz.1’

ldcc-20140209.tar.g 100%[===================>]   8.44M   558KB/s    in 11s     

2018-07-31 11:10:10 (761 KB/s) - ‘ldcc-20140209.tar.gz.1’ saved [8855190/8855190]



In [0]:
!tar zxf ldcc-20140209.tar.gz > /dev/null

In [0]:
!cat text/README.txt

livedoor ニュースコーパス


概要
-----------------
本コーパスは、NHN Japan株式会社が運営する「livedoor ニュース」のうち、下記のクリエイティブ・コモンズ
ライセンスが適用されるニュース記事を収集し、可能な限りHTMLタグを取り除いて作成したものです。

- トピックニュース
  http://news.livedoor.com/category/vender/news/
- Sports Watch
  http://news.livedoor.com/category/vender/208/
- ITライフハック
  http://news.livedoor.com/category/vender/223/
- 家電チャンネル
  http://news.livedoor.com/category/vender/kadench/
- MOVIE ENTER
  http://news.livedoor.com/category/vender/movie_enter/
- 独女通信
  http://news.livedoor.com/category/vender/90/
- エスマックス
  http://news.livedoor.com/category/vender/smax/
- livedoor HOMME
  http://news.livedoor.com/category/vender/homme/
- Peachy
  http://news.livedoor.com/category/vender/ldgirls/

収集時期：2012年9月上旬


ライセンス
-----------------
各記事ファイルにはクリエイティブ・コモンズライセンス「表示 - 改変禁止」
（http://creativecommons.org/licenses/by-nd/2.1/jp/）が適用されます。
クレジット表示についてはニュースカテゴリにより異なるため、サブディレクトリにある
それぞれの LICENSE.txt をご覧ください。

livedoor はNHN Japan株式会社の登録商標です。


記事ファイルのフォーマット
-----------------
記事ファイルは以下のフォーマットにしたが

カテゴリとテキストが、どのようなディレクトリやファイルになっているかを確認する。

In [0]:
!ls ./text

CHANGES.txt	kaden-channel	peachy	    sports-watch
dokujo-tsushin	livedoor-homme	README.txt  topic-news
it-life-hack	movie-enter	smax


In [0]:
!ls ./text/sports-watch/ | head

LICENSE.txt
sports-watch-4597641.txt
sports-watch-4601248.txt
sports-watch-4604621.txt
sports-watch-4609913.txt
sports-watch-4612295.txt
sports-watch-4623406.txt
sports-watch-4623588.txt
sports-watch-4627799.txt
sports-watch-4628232.txt
ls: write error: Broken pipe


- １行目：記事のURL
- ２行目：記事の日付
- ３行目：記事のタイトル
- ４行目以降：記事の本文

In [0]:
!head ./text/sports-watch/sports-watch-4597641.txt

http://news.livedoor.com/article/detail/4597641/
2010-02-10T10:50:00+0900
【Sports Watch】秋山成勲、メールで吉田に対戦迫った!?
今月8日、都内ホテルでは、総合格闘家・吉田秀彦の引退試合興行「ASTRA」の開催が発表された。

バルセロナ五輪柔道金メダリストとしての実績を引っさげ、2002年にプロ総合格闘家に転向。以後、数々の死闘を繰り広げてきた吉田。昨年大晦日のDynamite!!では、石井慧との金メダリスト対決を制し、4月に迎える引退試合の相手には、桜庭和志やまさかの朝青龍といった報道が駆け巡る中、“反骨の柔道王”秋山成勲が吉田にメールで対戦を迫っていたというのだ。

会見翌日の9日に更新された秋山成勲オフィシャルブログでは、「吉田秀彦対秋山成勲」と題し、「常に憧れ目標にしてきた吉田先輩が引退。正直寂しい気持ちはありますが、ほんまにお疲れ様でした！引退試合はもちろん自分とやるでしょ！？」とストレートに書き綴りながらも、「さっき吉田先輩にメールで自分とやるでしょ？的なメールを打って、軽く流されましたが」と、実際に吉田へ対戦を打診をしていたことを明かした。

もちろん、階級が違う上、秋山は米UFCを主戦場にしている現状、対戦することはまずない。それでも、我が道を行き、空気を一切読まない秋山だけに、案外本気に考えていた可能性もなきにしもあらず——。そんなブログの最後には、「ほんまに柔道から格闘技の道を作って頂いたパイオニアだと自分は思い尊敬してます！引退試合頑張ってください！」と吉田にエールを送る秋山であった。


In [0]:
!ls text/sports-watch | grep ".txt" | wc -l

901


## Pythonで文書分類を実装する

### データの前処理

#### 利用するモジュールのimport
- ファイルを読み込むためのglob
- データ整形を行うためのpandas
- 形態素解析器
- scikit-learn
  - CountVectorizer
  - TFIDFモジュール

In [0]:
import glob
import re

import MeCab
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [0]:
# LabelEncoderでどうしてもWarningが出るので、いったんWarningを非表示にする
import warnings

warnings.simplefilter("ignore")

#### 文書を読み込む
カテゴリは9カテゴリ

記事ファイルのフォーマットは以下であるため、これをタプル化する。

- １行目：記事のURL
- ２行目：記事の日付
- ３行目：記事のタイトル
- ４行目以降：記事の本文

In [0]:
categories = [
    "sports-watch", "topic-news", "dokujo-tsushin", "peachy",
    "movie-enter", "kaden-channel", "livedoor-homme", "smax",
    "it-life-hack",
]

In [0]:
docs = []
for category in categories:
    for f in glob.glob(f"./text/{category}/{category}*.txt"):
        # 1ファイルごとに処理
        with open(f, "r") as fin:
            # nextで1行取得する(__next__を呼ぶ)
            url = next(fin).strip()
            date = next(fin).strip()
            title = next(fin).strip()
            body = "\n".join([line.strip() for line in fin if line.strip()])
      
        docs.append((category, url, date, title, body))

In [0]:
docs[0]

('sports-watch',
 'http://news.livedoor.com/article/detail/4865712/',
 '2010-07-05T07:30:00+0900',
 '【Sports Watch】長谷部に対し、松井“キャプテンぶってる”',
 '3日放送の日本テレビ「Going! Sports＆News」では、1日にW杯の大舞台から帰国した日本代表の阿部勇樹（浦和レッドダイヤモンズ）、長谷部誠（ヴォルフスブルク）、長友佑都（FC東京）が生出演を果たした。\n冒頭、長友の大ファンだというマツコ・デラックスから「理想を100とすれば、限りなく90代後半。なぜ、この人がサッカー選手、それも日本代表に選ばれるような。そんな人がどうこうなるわけないじゃない。もしかしたら、日本を離れるかもしれないなんて話もあるから。話し相手に行くわよ」といったVTRが届く。これに対し、「どうなりたいんすかね？きっちいっすね」と苦笑いを浮かべた長友に、長谷部は「ホント、うらやましいですね」と他人ごとのように話す。\nまた、松井大輔がVTRで登場し、3人の素顔を明かすと、長谷部には「キャプテンぶってる!!」とフリップに書き、「アイツは、ぶってるね。この一言しかいわない」とニヤリ。阿部には「本当は駒ちゃんと○○」と書き、「仲いいからね。友情を超えたものがあるんじゃないか」と意味深な発言、長友には「マツコ・デラックスと（ハートのマーク）」と書いた。\n思わぬところで、駒野との仲を指摘された阿部は、「そんなんじゃないですよ。でも、中学から知ってるんで。松井は小学校から知ってる」と反論、“キャプテンぶってる”といわれた長谷部は、「自分でも名ばかりのキャプテンだったと思いますけど、松井さんに言われるとムカつきますね」と語り、長友も「いや、もうぶってますし。ただ“俺についてこい”ばりの背中をみせてくれました」と長谷部をフォローした。')

#### 取り扱いやすいデータフレーム形式に変換

In [0]:
df = pd.DataFrame(
        docs,
        columns=["category", "url", "date", "title", "body"],
        dtype="category"
)
# 日付は日付型に変更（今回使うわけでは無い）
df["date"] = pd.to_datetime(df["date"])

#### データの確認

In [0]:
# 先頭行数行を表示
df.head()

,category,url,date,title,body
0,sports-watch,http://news.livedoor.com/article/detail/4865712/,2010-07-04 22:30:00,【Sports Watch】長谷部に対し、松井“キャプテンぶってる”,3日放送の日本テレビ「Going! Sports＆News」では、1日にW杯の大舞台から帰国...
1,sports-watch,http://news.livedoor.com/article/detail/5575653/,2011-05-21 23:00:00,【Sports Watch】シンクロ美人姉妹の驚くべき特技とは？,朝日放送「Oh！どや顔サミット」には、元シンクロナイズドスイミング日本代表で、世界選手権にも...
2,sports-watch,http://news.livedoor.com/article/detail/4762254/,2010-05-11 02:23:00,【Sports Watch】テリー伊藤「相当、谷さんは図々しい」,10日、国民的アスリートとして知られる柔道家・谷亮子が、民主党から参院選比例代表に立候補する...
3,sports-watch,http://news.livedoor.com/article/detail/6359686/,2012-03-12 00:15:00,「無責任すぎる」元浦和フィンケ辞任に批判,10日、浦和レッズの元監督であるフォルカー・フィンケ氏が、1FCケルン（ドイツ）のスポーツデ...
4,sports-watch,http://news.livedoor.com/article/detail/6741928/,2012-07-11 00:15:00,長友が語る本田圭佑の面白さ「分かってないんです、周りの反応が」,10日放送、TBS「SUPER SOCCER」では、「新たなる旅立ちSP！長友佑都・加藤浩次...


In [0]:
# それぞれの要素数を数え上げる
df.category.value_counts()

sports-watch      900
smax              870
movie-enter       870
it-life-hack      870
dokujo-tsushin    870
kaden-channel     864
peachy            842
topic-news        770
livedoor-homme    511
Name: category, dtype: int64

#### 単語分割
MeCabで分かち書きを行う処理を関数化する。

DataFrameに分かち書き済みのテキストを格納する。

In [0]:
tagger = MeCab.Tagger("-Owakati")

def parse_to_wakati(text):
    # wakatiモードのMeCabのparseは、文字列を返す
    return tagger.parse(text).strip()

In [0]:
# dfのbody_wakatiという列に対して、bodyを分かち書きしたものを代入する
df = df.assign(body_wakati=df.body.apply(parse_to_wakati))

In [0]:
df.head()

,category,url,date,title,body,body_wakati
0,sports-watch,http://news.livedoor.com/article/detail/4865712/,2010-07-04 22:30:00,【Sports Watch】長谷部に対し、松井“キャプテンぶってる”,3日放送の日本テレビ「Going! Sports＆News」では、1日にW杯の大舞台から帰国...,3 日 放送 の 日本テレビ 「 Going ! Sports ＆ News 」 で は 、...
1,sports-watch,http://news.livedoor.com/article/detail/5575653/,2011-05-21 23:00:00,【Sports Watch】シンクロ美人姉妹の驚くべき特技とは？,朝日放送「Oh！どや顔サミット」には、元シンクロナイズドスイミング日本代表で、世界選手権にも...,朝日放送 「 Oh ！ ど や 顔 サミット 」 に は 、 元 シンクロナイズドスイミング...
2,sports-watch,http://news.livedoor.com/article/detail/4762254/,2010-05-11 02:23:00,【Sports Watch】テリー伊藤「相当、谷さんは図々しい」,10日、国民的アスリートとして知られる柔道家・谷亮子が、民主党から参院選比例代表に立候補する...,10 日 、 国民 的 アスリート として 知ら れる 柔道 家 ・ 谷 亮子 が 、 民主...
3,sports-watch,http://news.livedoor.com/article/detail/6359686/,2012-03-12 00:15:00,「無責任すぎる」元浦和フィンケ辞任に批判,10日、浦和レッズの元監督であるフォルカー・フィンケ氏が、1FCケルン（ドイツ）のスポーツデ...,10 日 、 浦和レッズ の 元 監督 で ある フォルカー・フィンケ 氏 が 、 1 FC...
4,sports-watch,http://news.livedoor.com/article/detail/6741928/,2012-07-11 00:15:00,長友が語る本田圭佑の面白さ「分かってないんです、周りの反応が」,10日放送、TBS「SUPER SOCCER」では、「新たなる旅立ちSP！長友佑都・加藤浩次...,10 日 放送 、 TBS 「 SUPER SOCCER 」 で は 、 「 新た なる 旅...


In [0]:
df.body_wakati.head()

0    3 日 放送 の 日本テレビ 「 Going ! Sports ＆ News 」 で は 、...
1    朝日放送 「 Oh ！ ど や 顔 サミット 」 に は 、 元 シンクロナイズドスイミング...
2    10 日 、 国民 的 アスリート として 知ら れる 柔道 家 ・ 谷 亮子 が 、 民主...
3    10 日 、 浦和レッズ の 元 監督 で ある フォルカー・フィンケ 氏 が 、 1 FC...
4    10 日 放送 、 TBS 「 SUPER SOCCER 」 で は 、 「 新た なる 旅...
Name: body_wakati, dtype: category
Categories (7359, object): [" ミスター " こと 長嶋 茂雄 ・ 終身 名誉 監督 の 発言 が 物議 を かもし て..., " 合コン 総研 アナリスト " の 水谷 麻衣 さん に 聞く 合コン の 極意 。 有名...,
                            " 女のコ の ため の 日本 最大 級 ファッションフェスタ "『 東京 ガールズコレクシ..., " 女のコ の ため の 日本 最大 級 ファッションフェスタ "『 東京 ガールズコレクシ...,
                            ..., ＮＴＴドコモ が 、 今年 の １２月 を めど に 、 イギリス の ボーダフォン 法人 ...,
                            ＳＦ 小説 の 巨匠 ロバート ・ Ａ ・ ハインライン の 傑作 『 宇宙 の 戦士 』 ..., ＷＢＡ 世界 バンタム 級 王者 ・ 亀田 興 毅 の ４ 度目 の 防衛 戦 が 、 ４月...,
                            ｉＰｈｏｎｅ ５ について 、 様々 な 憶測 が 飛び交っ て いる 。 早く 詳細 を ...]

#### ルールベースのためのラベルのエンコード
scikit-learnでは予測値のラベルは数値にエンコードする必要がある。

まず、ルールを書くために、予測するラベルをエンコードしておく

In [0]:
# ラベルエンコーダは、ラベルを数値に変換する
le = LabelEncoder()

# ラベルをエンコードし、エンコード結果をyに代入する
y = le.fit_transform(df.category)

In [0]:
y[:10]

array([7, 7, 7, 7, 7, 7, 7, 7, 7, 7])

In [0]:
# ラベルエンコーダがエンコードしたクラスの種類を表示
print(le.classes_)

# sports-watchクラスが、何にエンコードされたか確認
print(le.transform(["sports-watch"]))

['dokujo-tsushin' 'it-life-hack' 'kaden-channel' 'livedoor-homme'
 'movie-enter' 'peachy' 'smax' 'sports-watch' 'topic-news']
[7]


#### 学習とテストに分けて、学習と推定を行う

In [0]:
X_train, X_test, y_train, y_test = train_test_split(
                                      df.body_wakati,  # 入力
                                      y, # 正解ラベル
                                      test_size=0.2, # テストデータのサイズ（全体の何割か）
                                      random_state=42, # シャッフルしたときの乱数の種を固定
                                      shuffle=True
)

### ルールベース

In [0]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.metrics import classification_report, confusion_matrix

#### ルールベース分類器の作成

In [0]:
# scikit-learnのBaseEstimatorとTransformerMixinを継承することで、scikit-learn互換のクラスを作りやすくする
class RulebasedEstimator(BaseEstimator, TransformerMixin):
    def __init__(self, label_encoder):
        self.le = label_encoder
  
    def fit(self, X, y):
        return self

    def predict(self, X):
        """ルールを記述"""
        result = []
        for text in X:
            pred = 0
            if re.search(r"(スポーツ|サッカー)", text):
                pred = self.le.transform(["sports-watch"])[0]
            elif re.search(r"(独女|悩み|ブライド|サプライズ)", text):
                pred = self.le.transform(["dokujo-tsushin"])[0]
            elif re.search(r"(Mac|PC|IT|インテル|SSD|CPU)", text):
                pred = self.le.transform(["it-life-hack"])[0]
            elif re.search(r"(電力|冷蔵庫|扇風機|家電|熱中症)", text):
                pred = self.le.transform(["kaden-channel"])[0]
            elif re.search(r"(店|店舗|マラソン|スポーツ)", text):
                pred = self.le.transform(["livedoor-homme"])[0]
            elif re.search(r"(映画|芸能|リアクション|公開)", text):
                pred = self.le.transform(["movie-enter"])[0]
            elif re.search(r"(食べ物|菓子|ごはん|ワイン|カフェ)", text):
                pred = self.le.transform(["peachy"])[0]
            elif re.search(r"(ゲーム|スマホ)", text):
                pred = self.le.transform(["smax"])[0]
            elif re.search(r"(メダル|プレー)", text):
                pred = self.le.transform(["sports-watch"])[0]
            elif re.search(r"(テレビ|ジャーナリスト)", text):
                pred = self.le.transform(["topic-news"])[0]

            result.append(pred)
        return result

#### ルールベースによる分類

In [0]:
rulebased = RulebasedEstimator(label_encoder=le)

In [0]:
rulebased_pred = rulebased.predict(X_test)

####ルールベースの評価

In [0]:
print(classification_report(y_test, rulebased_pred, target_names=le.classes_))

                precision    recall  f1-score   support

dokujo-tsushin       0.19      0.45      0.27       176
  it-life-hack       0.42      0.59      0.49       139
 kaden-channel       0.83      0.30      0.44       178
livedoor-homme       0.12      0.23      0.16        83
   movie-enter       0.52      0.76      0.62       182
        peachy       0.38      0.05      0.09       173
          smax       0.25      0.09      0.13       182
  sports-watch       0.56      0.57      0.57       191
    topic-news       0.21      0.11      0.14       170

   avg / total       0.41      0.36      0.33      1474



  | 実際は正 | 実際は負
--- | -- | --
予測は正 | True Positive | False Positive
予測は負 | False Negative | True Negative

- 適合率(Precision)：正と予測したうち、実際に正であるものの割合

$$ Precision = \frac{True Positive}{True Positive + False Positive} $$

- 再現率(Recall)：実際に正であるもののうち、正と予測されたものの割合

$$ Recall = \frac{True Positive}{True Positive + False Negative} $$

- F値(F-measure)：適合率と再現率の調和平均

$$ F-measure = \frac{2 \cdot Recall \cdot Precision}{Recall + Precision} $$

### 多変数ベルヌーイモデルを用いたナイーブベイズ分類器

データの入力から推定までのパイプラインを作成する。

単語の頻度を数え上げ、それを分類器が学習するというパイプラインを作る。

In [0]:
text_clf = Pipeline([
    ("count_vec", CountVectorizer()),
    ("clf", MultinomialNB()),
])

作成したパイプラインに学習データを流し込む。

In [0]:
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('count_vec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

テストデータを入力し、分類する。

In [0]:
pred = text_clf.predict(X_test)

評価する。

In [0]:
print(classification_report(y_test, pred, target_names=le.classes_))

                precision    recall  f1-score   support

dokujo-tsushin       0.79      0.90      0.84       176
  it-life-hack       0.89      0.91      0.90       139
 kaden-channel       0.96      0.90      0.93       178
livedoor-homme       0.80      0.63      0.70        83
   movie-enter       0.94      0.98      0.96       182
        peachy       0.88      0.81      0.84       173
          smax       0.94      0.99      0.97       182
  sports-watch       0.94      0.98      0.96       191
    topic-news       0.95      0.87      0.91       170

   avg / total       0.90      0.90      0.90      1474



In [0]:
print(confusion_matrix(y_test, pred, labels=le.transform(le.classes_)))

[[158   0   2   1   2  11   0   2   0]
 [  2 126   1   1   0   0   6   0   3]
 [  2   7 160   3   0   0   4   0   2]
 [  9   5   1  52   5   8   2   0   1]
 [  2   0   0   0 179   1   0   0   0]
 [ 23   0   0   5   4 140   0   1   0]
 [  0   1   0   0   0   0 181   0   0]
 [  0   0   0   1   0   0   0 188   2]
 [  5   3   2   2   1   0   0   9 148]]


#### 任意のテキストを入力して解析する

In [0]:
original_text = "あの「Orange is the new black」のシーズン６が7月28日についにNetflixに上陸！"

wakati_text = parse_to_wakati(original_text)
print(le.inverse_transform(text_clf.predict([wakati_text]))[0])

movie-enter


#### 間違った例を見る

In [0]:
idx = y_test != pred
for text, t, p in zip(X_test[idx][:5], y_test[idx][:5], pred[idx][:5]):
    print("予測：{}\n推定:{}".format(le.inverse_transform(t), le.inverse_transform(p)))
    print(text)

予測：dokujo-tsushin
推定:peachy
今年 8 月末 に 、 イギリス の 経済 誌 「 エコノミスト 」 を 発行 する エコノミスト ・ インテリジェンス ・ ユニット による 「 世界 の 住み やすい 都市 ランキング 」 の 2011 年版 が 発表 さ れ た 。 これ は 、 世界 の 140 都市 の 「 住み やす さ 」 を 、 「 安定 性 」 、 「 ヘルスケア 」 、 「 文化 と 環境 」 、 「 教育 」 、 「 インフラ 」 など を 含む 30 項目 について 評価 し て 数値 化 する という もの で 、 今年 、 メルボルン が 初めて 1 位 に 輝い た 。 また 、 6 位 に シドニー 、 8 位 に パース と アデレード が 入り 、 オーストラリア の 都市 が トップ 10 以内 に 4 つも ランク イン し 、 オーストラリア が 「 世界 で もっとも 住み やすい 国 」 で ある こと が 証明 さ れ た 。 当然 ながら 観光 都市 として も 最適 で ある こと は 言うまでもなく 、 とくに 日本 が 秋 から 冬 に 向かう これから の 季節 は 、 南半球 の オーストラリア は 春 から 夏 に 向かう 、 まさに 旅行 の ベスト シーズン と なる 時期 だ 。 そこで カンタス航空 で は 、 観光 の ベスト シーズン を 迎える オーストラリア 南部 へ の 旅行 を 対象 に 、 11 月 1 日 〜 12 月 10 日 の 期間 に 成田 ⇔ シドニー 、 メルボルン 、 ブリスベン 、 アデレード 、 ホバート の いずれ か 、 羽田 ⇔ パース （ シンガポール 経由 ） の エコノミー 往復 を 7 万 2000 円 （ ※ ） にて ご 提供 する キャンペーン を 実施 する 。 時差 が 1 〜 2 時間 と 少ない オーストラリア へ は 、 日本 を 夜 出発 すれ ば 機内 で ゆっくり 休ん で 、 翌朝 オーストラリア に 到着 。 また 帰り は 日本 に 早朝 到着 する ので 、 週末 や 短い 休暇 で も 現地 で の 滞在 時間 を 最大限 楽しめる ところ も 大きな 魅力 の ひとつ だ 。 （ ※ 燃油 特別

### Bernoulliのナイーブベイズ分類器
頻度の数え上げを行い、ナイーブベイズ分類器を学習する。

$$ P(x_{i} | y) = P(i | y)x_{i} + (1-P(i|y))(1-x_{i})$$

In [0]:
bernoulli_clf = Pipeline([
    ("count_vec", CountVectorizer()),
    ("clf", BernoulliNB()),
])

作成したパイプラインに基づき学習する。

In [0]:
bernoulli_clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('count_vec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('clf', BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True))])

学習したパラメータを用いて、クラス推定を行う。

In [0]:
pred = bernoulli_clf.predict(X_test)
print(classification_report(y_test, pred, target_names=le.classes_))

                precision    recall  f1-score   support

dokujo-tsushin       0.83      0.90      0.86       176
  it-life-hack       0.91      0.92      0.91       139
 kaden-channel       0.89      0.95      0.92       178
livedoor-homme       0.91      0.47      0.62        83
   movie-enter       0.96      0.93      0.94       182
        peachy       0.85      0.81      0.83       173
          smax       0.99      1.00      0.99       182
  sports-watch       0.85      0.98      0.91       191
    topic-news       0.94      0.89      0.92       170

   avg / total       0.90      0.90      0.90      1474



### 線形SVMによる分類器

In [0]:
from sklearn.svm import LinearSVC

In [0]:
svm_clf = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", LinearSVC()),
])

svm_clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [0]:
pred = svm_clf.predict(X_test)
print(classification_report(y_test, pred, target_names=le.classes_))

                precision    recall  f1-score   support

dokujo-tsushin       0.89      0.89      0.89       176
  it-life-hack       0.95      0.96      0.95       139
 kaden-channel       0.98      0.96      0.97       178
livedoor-homme       0.90      0.78      0.84        83
   movie-enter       0.97      0.99      0.98       182
        peachy       0.92      0.91      0.92       173
          smax       0.99      1.00      1.00       182
  sports-watch       0.93      0.98      0.95       191
    topic-news       0.96      0.94      0.95       170

   avg / total       0.95      0.95      0.95      1474



### ランダムフォレストによる分類器

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
rf_clf = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", RandomForestClassifier()),
])

rf_clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [0]:
pred = rf_clf.predict(X_test)
print(classification_report(y_test, pred, target_names=le.classes_))

                precision    recall  f1-score   support

dokujo-tsushin       0.75      0.88      0.81       176
  it-life-hack       0.84      0.90      0.87       139
 kaden-channel       0.88      0.92      0.90       178
livedoor-homme       0.70      0.40      0.51        83
   movie-enter       0.88      0.92      0.90       182
        peachy       0.84      0.74      0.79       173
          smax       0.98      0.99      0.99       182
  sports-watch       0.86      0.93      0.90       191
    topic-news       0.94      0.83      0.88       170

   avg / total       0.86      0.86      0.86      1474



### 多層パーセプトロンによる分類器

学習に数分時間が必要

In [0]:
from sklearn.neural_network import MLPClassifier

In [0]:
mlp_clf = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MLPClassifier()),
])

mlp_clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False))])

In [0]:
pred = mlp_clf.predict(X_test)
print(classification_report(y_test, pred, target_names=le.classes_))

                precision    recall  f1-score   support

dokujo-tsushin       0.94      0.91      0.92       176
  it-life-hack       0.98      0.96      0.97       139
 kaden-channel       0.98      0.93      0.95       178
livedoor-homme       0.87      0.83      0.85        83
   movie-enter       0.88      0.99      0.93       182
        peachy       0.91      0.84      0.87       173
          smax       0.97      0.99      0.98       182
  sports-watch       0.95      0.99      0.97       191
    topic-news       0.94      0.92      0.93       170

   avg / total       0.94      0.94      0.94      1474



### 多変数ベルヌーイモデルをnumpyで実装
まだfor文があり、不完全な実装で動作が遅いです。

In [0]:
import numpy as np

from scipy.sparse import lil_matrix, csr_matrix


class NaiveBayes(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.vocabularies = None
        self.categories = None
        self.category_count = None
        self.category_word_occurence = None
        self.denominator = {}
    
    def fit(self, X, y):
        self.vocabularies = np.unique(X)
        self.categories, self.category_count = np.unique(y, return_counts=True)
        
        self.category_word_occurrence = lil_matrix((self.categories.shape[0], X.shape[1]))
        for c in self.categories:
            _y = y[y == c]
            _X = X[y == c]
            self.category_word_occurrence[c] = np.sum(_X, axis=0)
        self.category_word_occurrence = self.category_word_occurrence.tocsr()
        
        # P(word | cat)の分母 Σ |(cat, word)| + |V|
        for cat in self.categories:
            self.denominator[cat] = np.sum(self.category_word_occurrence[cat]) + len(self.vocabularies)
    
    def word_prob(self, word, category):
        """P(word|cat)"""
        return (self.category_word_occurrence[category, word]+1) / self.denominator[category]
    
    def score(self, doc, cat):
        """P(doc|cat)"""
        sc = np.log(self.category_count[cat] / self.category_count.sum())
        for word in doc.nonzero()[1]:
            sc += np.log(self.word_prob(word, cat))
        return sc
    
    def predict(self, X):
        return np.array([self._predict(doc) for doc in X])
    
    def _predict(self, doc):
        """P(cat|doc)"""
        return np.argmax(np.array([self.score(doc, cat) for cat in self.categories]))

In [0]:
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)

In [0]:
nb_clf = NaiveBayes()
nb_clf.fit(X_train_vec, y_train)

In [0]:
pred = nb_clf.predict(vectorizer.transform(X_test))
print(classification_report(y_test, pred, target_names=le.classes_))

## ルールベースの改善をしてみよう

In [0]:
df.query("category == 'smax'").head()

,category,url,date,title,body,body_wakati
5627,smax,http://news.livedoor.com/article/detail/6903954/,2012-08-30 05:55:00,Sony Mobile、端末下部にイルミネーションを備えた低価格帯モデル「Xperia J」を発表,ゼロから始めるスマートフォン\nソニーモバイルコミュニケーションズは29日（現地時間）、ドイ...,ゼロ から 始める スマート フォン ソニーモバイルコミュニケーションズ は 29 日 （ ...
5628,smax,http://news.livedoor.com/article/detail/6559474/,2012-05-15 01:35:00,KDDI、au向けWiMAX対応Android 4.0 ICS搭載「ARROWS Z ISW...,クアッドコアCPU搭載スマートフォンが登場！\nKDDIおよび沖縄セルラーは15日、この夏に...,クアッドコア CPU 搭載 スマートフォン が 登場 ！ KDDI および 沖縄 セルラー ...
5629,smax,http://news.livedoor.com/article/detail/6536595/,2012-05-07 22:55:00,ネコ好きはすぐにインストールしよう！天気、バッテリー残量、時間をニヤニヤしながらチェックでき...,とにかくキュートで可愛い名前がないネコ！\nライブ壁紙はいろいろな種類がありますが、キュート...,とにかく キュート で 可愛い 名前 が ない ネコ ！ ライブ 壁紙 は いろいろ な 種...
5630,smax,http://news.livedoor.com/article/detail/6865709/,2012-08-18 00:55:00,ちっちゃカワイイ仕草がぱないの！Android向けデスクトップマスコットLiplisに「忍野...,Android対応のデスクトップマスコットLiplisに＜物語＞シリーズのキャラクター忍野忍...,Android 対応 の デスク トップ マスコット Liplis に ＜ 物語 ＞ シリー...
5631,smax,http://news.livedoor.com/article/detail/6581246/,2012-05-22 00:55:00,東京新名所誕生！「東京スカイツリー周辺散策ガイド（Getting around TokyoS...,スカイツリー周辺ガイドでチェックしよう！\n本日22日（火）に東京スカイツリーがOPENしま...,スカイ ツリー 周辺 ガイド で チェック しよ う ！ 本日 22 日 （ 火 ） に 東...


In [0]:
# scikit-learnのBaseEstimatorとTransformerMixinを継承することで、scikit-learn互換のクラスを作りやすくする
class RulebasedEstimator(BaseEstimator, TransformerMixin):
    def __init__(self, label_encoder):
        self.le = label_encoder
  
    def fit(self, X, y):
        return self

    def predict(self, X):
        """ルールを記述"""
        result = []
        for text in X:
            pred = 0
            if re.search(r"(スポーツ|サッカー)", text):
                pred = self.le.transform(["sports-watch"])[0]
            elif re.search(r"(独女|悩み|ブライド|サプライズ)", text):
                pred = self.le.transform(["dokujo-tsushin"])[0]
            elif re.search(r"(Mac|PC|IT|インテル|SSD|CPU)", text):
                pred = self.le.transform(["it-life-hack"])[0]
            elif re.search(r"(電力|冷蔵庫|扇風機|家電|熱中症)", text):
                pred = self.le.transform(["kaden-channel"])[0]
            elif re.search(r"(店|店舗|マラソン|スポーツ)", text):
                pred = self.le.transform(["livedoor-homme"])[0]
            elif re.search(r"(映画|芸能|リアクション|公開)", text):
                pred = self.le.transform(["movie-enter"])[0]
            elif re.search(r"(食べ物|菓子|ごはん|ワイン|カフェ)", text):
                pred = self.le.transform(["peachy"])[0]
            elif re.search(r"(ゲーム|スマホ)", text):
                pred = self.le.transform(["smax"])[0]
            elif re.search(r"(メダル|プレー)", text):
                pred = self.le.transform(["sports-watch"])[0]
            elif re.search(r"(テレビ|ジャーナリスト)", text):
                pred = self.le.transform(["topic-news"])[0]

            result.append(pred)
        return result

In [0]:
rulebased = RulebasedEstimator(label_encoder=le)
rulebased_pred = rulebased.predict(X_test)
print(classification_report(y_test, rulebased_pred, target_names=le.classes_))